# Lab 1:  Data Loading, GPU Dataframe Creation, and Data Manipulation
Thanks to Anaconda for some material

In this lab you will learn how to 
- Load data into a GPU Data Frame (GDF)
- Learn about Dataframe level data functions
- Learn about seriers level data function 
- Manitulate data in a GDF to performance some basic ETL and statistical functions


For this lab we will be looking at Netflow data.  The data consist of seven column which make it easy to visually inspect.  We will be manipulating the data in various way, many of which make not sense for doing cyber analysis.  


This lab should take 50 minutes


<br>
As you progress in this lab look for instances of ***TASK:***, this will be where you will be asked to take an action to complete this lab.

***TASK:*** Execute the cell below to auto-time execution of every cell

In [1]:
# Add autotime of each block
!pip install ipython-autotime
%load_ext autotime

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


<br>
_____
# Loading Data
The first file we are going to load is netflow1.csv
  
Size = 965 MB<br>
Records = 17,296,829

<p>
### Load using the traditional Pandas interface

In [2]:
import pandas as pd

time: 197 ms


In [3]:
# let's define the data - column names and data types
cols = [
    "strdate",
    "srcip",
    "dstip",
    "srcport",
    "dstport",
    "srcbytes",
    "dstbytes"   
]


dtypes = {    
    "strdate"  : str,
    "srcip"    : str,
    "dstip"    : str,
    "srcport"  : int,
    "dstport"  : int,
    "srcbytes" : int,
    "dstbytes" : int

}

time: 4.84 ms


In [4]:
file_1 = '../data/netflow/netflow1.csv'

time: 1.32 ms


In [5]:
# the data file contains a header line that needs to be skipped
df = pd.read_csv(file_1,  names=cols, dtype=dtypes, skiprows=1)

time: 10.2 s


In [6]:
df.dtypes

strdate     object
srcip       object
dstip       object
srcport      int64
dstport      int64
srcbytes     int64
dstbytes     int64
dtype: object

time: 13.7 ms


In [7]:
df.head()

,strdate,srcip,dstip,srcport,dstport,srcbytes,dstbytes
0,2013-04-01 07:50:16,172.20.0.3,172.255.255.255,137,137,1104,0
1,2013-04-01 07:50:21,172.10.0.40,172.255.255.255,137,137,184,0
2,2013-04-01 08:05:00,172.10.1.17,10.0.0.13,5040,80,454,633
3,2013-04-01 08:05:01,172.10.1.9,10.1.0.77,5067,80,454,633
4,2013-04-01 08:05:01,172.30.2.114,10.0.0.9,8749,80,453,632


time: 15.6 ms


<br>
### Creating a GPU Dataframe


In [8]:
import pygdf

time: 317 ms


In [9]:
gdf = pygdf.DataFrame.from_pandas(df)

time: 957 ms


In [10]:
gdf.dtypes

strdate     object
srcip       object
dstip       object
srcport      int64
dstport      int64
srcbytes     int64
dstbytes     int64
dtype: object

time: 5.8 ms


***Congrats you have craeted a GPU Data Frame***

In [11]:
# let's look at the data
gdf.head()

              strdate        srcip           dstip srcport dstport srcbytes dstbytes
0 2013-04-01 07:50:16   172.20.0.3 172.255.255.255     137     137     1104        0
1 2013-04-01 07:50:21  172.10.0.40 172.255.255.255     137     137      184        0
2 2013-04-01 08:05:00  172.10.1.17       10.0.0.13    5040      80      454      633
3 2013-04-01 08:05:01   172.10.1.9       10.1.0.77    5067      80      454      633
4 2013-04-01 08:05:01 172.30.2.114        10.0.0.9    8749      80      453      632

time: 23.4 ms


### Transitioning GDF back to Pandas

In [12]:
# just as easy
df2 = gdf.to_pandas()

time: 1.14 s


In [13]:
df2.dtypes

strdate     object
srcip       object
dstip       object
srcport      int64
dstport      int64
srcbytes     int64
dstbytes     int64
dtype: object

time: 5.06 ms


### Clean-up: deleting a dataframe

In [14]:
# clean up since we don't need df2
del df2

time: 123 ms


In [15]:
# Also drop the GPU dataframe since we will recreate it with some options
del gdf

time: 1.08 ms


In [18]:
del df

time: 1.27 ms


# Column-based funtions and Data Manipulation

In [41]:
# reload the data
# the data file contains a header line that needs to be skipped
df = pd.read_csv(file_1,  names=cols, dtype=dtypes, skiprows=1)

time: 10.1 s


### Limitations on the current GDF version
- Date are not supported   => scheduke for next version
- String are not supported => also schedule for next version

In [42]:
df.strdate.head(2)

0    2013-04-01 07:50:16
1    2013-04-01 07:50:21
Name: strdate, dtype: object

time: 6.37 ms


In [43]:
# Within Pandas - Convert the data from a string into parts
df['Date']  = pd.to_datetime(df['strdate'], format='%Y-%m-%d %H:%M:%S')

time: 2.82 s


In [44]:
df['Year']  = df.Date.dt.year
df['Month'] = df.Date.dt.month

time: 1.15 s


***Task:***  Convert day, hour, min, and second
<details><summary>Click for Answer</summary>
<p>
df['Day']  = df.Date.dt.day
df['Hour'] = df.Date.dt.hour
df['Minute']  = df.Date.dt.minute
df['Second'] = df.Date.dt.second
</p>
</details>

In [45]:
#remove
df['Day']  = df.Date.dt.day
df['Hour'] = df.Date.dt.hour
df['Minute']  = df.Date.dt.minute
df['Second'] = df.Date.dt.second

time: 2.28 s


In [46]:
df.dtypes

strdate             object
srcip               object
dstip               object
srcport              int64
dstport              int64
srcbytes             int64
dstbytes             int64
Date        datetime64[ns]
Year                 int64
Month                int64
Day                  int64
Hour                 int64
Minute               int64
Second               int64
dtype: object

time: 5.46 ms


In [47]:
# drop the columns we no longer care about
df = df.drop(columns=['strdate','Date'])

time: 2.76 s


In [48]:
# now create the GDF one more time
gdf = pygdf.DataFrame.from_pandas(df)

time: 1.13 s


<br>
That's it - the data is now loaded and available for accelerated Data Analysis on the GPU
<p>
*Rember that **df = CPU** and **gdf = GPU** since we will switch back and forth for performance comparisons*<p>
***Data Loading Runtime*** = pd.read_csv + DataFrame.from_pandas

<br>
# Column Functionals and Transformations
One of the basic GDF operations is column transform. To do that we use built-in arithmetic operations on each column

***Note:*** The followiong function operate against a GDF Column - not against the full dataset.


### Simple Counting

In [49]:
# How many 'Non-NULL' records are in the dataset?
gdf['dstbytes'].count()

17296828

time: 4.02 ms


In [50]:
# First some simple counting of the number of rows
gdf['srcport'].count()

17296828

time: 3.53 ms


***Task:***  How long does it take to do that simple counting on the CPU?
<details><summary>Click for Answer</summary>
<p>
df['dstbytes'].count()
</p>
</details>

In [52]:
#Remove
df['dstbytes'].count()

17296828

time: 49.4 ms


In [53]:
### That is not enough data to truely push a GPU, let's load more
file_2 = '../data/netflow/netflow1.csv'
file_3 = '../data/netflow/netflow1.csv'
file_4 = '../data/netflow/netflow1.csv'

df2 = pd.read_csv(file_2,  names=cols, dtype=dtypes, skiprows=1)
df3 = pd.read_csv(file_3,  names=cols, dtype=dtypes, skiprows=1)
df4 = pd.read_csv(file_4,  names=cols, dtype=dtypes, skiprows=1)

time: 30.5 s


In [56]:
def ExtractDateTeime (raw_df) :
    raw_df['Date']   = pd.to_datetime(raw_df['strdate'], format='%Y-%m-%d %H:%M:%S')
    raw_df['Year']   = raw_df.Date.dt.year
    raw_df['Month']  = raw_df.Date.dt.month    
    raw_df['Day']    = raw_df.Date.dt.day
    raw_df['Hour']   = raw_df.Date.dt.hour
    raw_df['Minute'] = raw_df.Date.dt.minute
    raw_df['Second'] = raw_df.Date.dt.second   
    
    raw_df = raw_df.drop(columns=['strdate','Date'])
    
    return raw_df

time: 12.7 ms


In [57]:
df2 = ExtractDateTeime(df2)
df3 = ExtractDateTeime(df3)
df4 = ExtractDateTeime(df4)

time: 26.5 s


In [58]:
gdf2 = pygdf.DataFrame.from_pandas(df2)
gdf3 = pygdf.DataFrame.from_pandas(df3)
gdf4 = pygdf.DataFrame.from_pandas(df4)

time: 3.52 s


In [59]:
# Concatenate the 4 GDF together
gdf = pygdf.concat([gdf, gdf2, gdf3, gdf4])

time: 57.1 ms


In [64]:
gdf['srcport'].count()

69187312

time: 3.23 ms


was 17,296,828 now is 69,187,312

In [61]:
# do the same on the CPU so that the two datasets are in sync
df = pd.concat([df, df2, df3, df4])

time: 4.33 s


## Data Type Conversion
Let's continue cleanign up the data

<br>
***Task***   Drop the scrip and dstip columns since string is not a supported data type
<br>
<br>
<details><summary>Click for answer</summary>
<p>
gdf.drop_column('srcip')
gdf.drop_column('dstip')
</p>
</details>

In [66]:
#Remove
gdf.drop_column('srcip')
gdf.drop_column('dstip')

time: 20.5 ms


### What is the max value for the source port?

In [68]:
gdf['srcport'].max()

65534

time: 9.8 ms


In [69]:
# Since the largest dstbyte size is less than an int32, let's convert the data type
import numpy as np
gdf['srcport'] = gdf['srcport'].astype(np.int32)

time: 117 ms


/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


In [70]:
gdf.dtypes

srcport     int32
dstport     int64
srcbytes    int64
dstbytes    int64
Year        int64
Month       int64
Day         int64
Hour        int64
Minute      int64
Second      int64
dtype: object

time: 5.76 ms


<br>
***Tasks*** Find the max values for dstport, srcbytes, and dstbyutes

<details><summary>Click for Answer</summary>
<p>
gdf['dstport'].max()
gdf['srcbytes'].max()
gdf['dstbytes'].max()
</p>
</details>

<br>
***Tasks*** Convert dstport to int32

<details><summary>Click for Answer</summary>
<p>
gdf['dstport'] = gdf['dstport'].astype(np.int32)
</p>
</details>

***Question*** Why do we care about using smaller data types?

<br>
***Tasks*** Looking at dstbytes:  what is min, max, mean, and standard deviation?

<details><summary>Click for Answer</summary>
<p>
gdf['dstbytes'].min()
gdf['dstbytes'].max()
gdf['dstbytes'].mean()
gdf['dstbytes'].std()
</p>
</details>

In [71]:
# Remove
gdf['dstbytes'].min()
gdf['dstbytes'].max()
gdf['dstbytes'].mean()
gdf['dstbytes'].std()

/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


22052.182972823728

time: 145 ms


## Now Let's create a new column

<br>
### Transformations
***Tasks*** Create a new GDF column called ***totalbyes*** that is the sum of src and dst bytes

In [73]:
gdf['totalbytes'] = gdf['srcbytes'] + gdf['dstbytes']

time: 26.5 ms


In [74]:
# Verify that a new column was created
gdf.dtypes

srcport       int32
dstport       int64
srcbytes      int64
dstbytes      int64
Year          int64
Month         int64
Day           int64
Hour          int64
Minute        int64
Second        int64
totalbytes    int64
dtype: object

time: 6.27 ms


In [75]:
# What is the max byte size
gdf['totalbytes'].max()

8094046

time: 9.77 ms


#### Let's try that same function on the CPU

In [76]:
# See the same time on the CPU
df['totalbytes'] = df['srcbytes'] + df['dstbytes']

time: 1.29 s


In [77]:
df['totalbytes'].max()

8094046

time: 534 ms


The performance gain is on a small dataset and a simple transformation.  As data size and analytic complexity increas so does the delta in performance.

# DataFrame Operations
The following function operate on a dataframe

### Filtering
Selecting a subregion of data that matches an expresion

In [78]:
# Let's get a count for reference of teh total number of records
gdf['srcport'].count()

69187312

time: 6.92 ms


### Extract a new Dataframe where the DST Port is not port 80

In [79]:
port_not_80 = gdf.query('dstport != 80')

/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


time: 587 ms


***Task:***  How many records are in the new set?
<details><summary>Click for Answer</summary>
<p>
port_not_80['srcport'].count()
</p>
</details>

In [80]:
#remove
port_not_80['srcport'].count()

3325024

time: 3.53 ms


<p>
### Grouping and Aggregations

In [81]:
from collections import OrderedDict

time: 1.33 ms


In [82]:
gdf.dtypes

srcport       int32
dstport       int64
srcbytes      int64
dstbytes      int64
Year          int64
Month         int64
Day           int64
Hour          int64
Minute        int64
Second        int64
totalbytes    int64
dtype: object

time: 6.13 ms


In [83]:
# add a column for count
gdf['count'] = gdf['dstport']
gdf.dtypes

srcport       int32
dstport       int64
srcbytes      int64
dstbytes      int64
Year          int64
Month         int64
Day           int64
Hour          int64
Minute        int64
Second        int64
totalbytes    int64
count         int64
dtype: object

time: 24.8 ms


In [84]:
aggs = OrderedDict()
aggs['count'] = 'count'

stats = gdf.groupby(['dstport']).agg(aggs)

/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home

time: 16 s


In [85]:
stats.dtypes

dstport      int64
count      float64
dtype: object

time: 4.07 ms


In [86]:
stats.head(25)

   dstport      count
 0       0   347708.0
 1      21      740.0
 2      25   312028.0
 3      80 65862288.0
 4     123   110712.0
 5     137     2900.0
 6     138     9108.0
 7     445        4.0
 8    1025       32.0
 9    1026       64.0
10    1027       56.0
11    1028       64.0
12    1029       64.0
13    1030       64.0
14    1031       68.0
15    1032       64.0
16    1033       68.0
17    1034       68.0
18    1035       60.0
19    1036       68.0
20    1037       60.0
21    1038       60.0
22    1039       72.0
23    1040       56.0
24    1041       84.0

time: 90.3 ms


In [87]:
del stats

time: 61.1 ms


***Tasks:  What is the count of SRC - DST Port groupings
<br>
<br>
<details><summary>Click for answer</summary>
<p>
aggs = OrderedDict()
aggs['count'] = 'count'

stats = gdf.groupby(['srcport','dstport']).agg(aggs)
</p>
</details> 

In [88]:
aggs = OrderedDict()
aggs['count'] = 'count'

stats = gdf.groupby(['srcport','dstport']).agg(aggs)

/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/home/brad/anaconda3/lib/python3.6/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


time: 45.4 s


In [89]:
stats.head(25)

   srcport dstport    count
 0       0       0 347708.0
 1      20    3098      4.0
 2      20    3099      4.0
 3      20    3100      4.0
 4      20    3103      4.0
 5      20    3544      4.0
 6      20    3545      4.0
 7      20    3546      4.0
 8      20    3547      4.0
 9      20    3548      4.0
10      20    3549      4.0
11      20    3552      4.0
12      20    3553      4.0
13      20    8403      4.0
14      20    8404      4.0
15      20    8405      4.0
16      20    8406      4.0
17      20    8407      4.0
18      20    8408      4.0
19      21    1048      4.0
20      21    1074      4.0
21      21    1084      4.0
22      21    1087      4.0
23      21    1137      4.0
24      21    1139      4.0

time: 17.2 ms


In [90]:
del stats

time: 1.23 ms


## Independent Task

***TASK***
- Normalize the total byte count to KB
- Compute the Mean and Standard Deviations
- Compute the Z-Score

Answer below - just one possibility

In [ ]:
gdf.dtypes

In [ ]:
def ScaleData( totalbytes, KB, Scale) :
    for i in range (totalbytes.size) :
        KB[i] = totalbytes[i] / Scale

In [ ]:
gdf = gdf.apply_rows(ScaleData, incols=['totalbytes'], outcols=dict(KB=np.float32), kwargs=dict(Scale=(1024)))

In [ ]:
gdf.head(25)

In [ ]:
mean = gdf['KB'].mean()

In [ ]:
std = gdf['KB'].std()

In [ ]:
print("Std == %f \t Mean == %f" % (std, mean))

In [ ]:
def NormalizeData( KB, Z, s, m) :
    for i in range(KB.size):
        Z[i] =  ( (KB[i] - m)/s )


In [ ]:
gdf = gdf.apply_rows(NormalizeData, incols=['KB'], outcols=dict(Z=np.float32), kwargs=dict(s=std, m=mean ))

In [ ]:
gdf.head(30)